# Semantic ID Recommender - Training Test

This notebook tests the full training pipeline in Google Colab.

**Requirements:**
- Google Colab with GPU runtime (T4 is sufficient)
- ~10 minutes for full test

**What this tests:**
1. RQ-VAE training (semantic ID learning)
2. Training data generation
3. LLM fine-tuning with Unsloth
4. Inference test

## Setup

In [ ]:
!nvidia-smi

In [ ]:
# Install uv
!pip install uv

# Clone private repo (skip if already exists)
import os
if not os.path.exists("semantic_id_recommender"):
    !git clone https://github.com/charleslow/semantic_id_recommender.git
else:
    print("✓ Repo already exists")
    
%cd semantic_id_recommender
!git pull --ff-only

In [ ]:
# Install dependencies (let uv resolve versions compatible with Colab's environment)
!uv pip install --system \
    torch torchvision torchaudio \
    transformers datasets accelerate \
    vector-quantize-pytorch sentence-transformers \
    lightning wandb omegaconf einops tqdm rich \
    pydantic

# Fix protobuf version conflict
!uv pip install --system "protobuf>=3.20,<5"

# Note: We skip using uv.lock because Colab has pre-installed packages
# that may conflict with locked versions

## 1. Import from Package

In [ ]:
# Add repo root to Python path for imports
import sys
sys.path.insert(0, ".")

# Import from the cloned package
import torch
from src.rqvae.model import SemanticRQVAE, SemanticRQVAEConfig
from src.rqvae.trainer import RQVAETrainer
from src.rqvae.dataset import ItemEmbeddingDataset

print("✓ Package imports successful")

## 2. Create Test Dataset

In [ ]:
from src.rqvae.dataset import create_dummy_catalogue, ItemEmbeddingDataset
import json

NUM_ITEMS = 500

# Create dummy catalogue
create_dummy_catalogue(num_items=NUM_ITEMS, output_path="data/catalogue.json")

# Load and inspect
with open("data/catalogue.json") as f:
    catalogue = json.load(f)
    items = catalogue["items"]

print(f"\nSample items:")
for item in items[:5]:
    print(f"  {item['id']}: {item['title']} - ${item['price']}")

## 3. Generate Embeddings

In [ ]:
# Use the package's ItemEmbeddingDataset to generate embeddings
dataset = ItemEmbeddingDataset.from_catalogue(
    catalogue_path="data/catalogue.json",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    cache_path="data/embeddings.pt",  # Cache for faster reruns
)

print(f"\nDataset info:")
print(f"  Number of items: {len(dataset)}")
print(f"  Embedding dim: {dataset.embeddings.shape[1]}")
print(f"  Item IDs (first 5): {dataset.item_ids[:5]}")

# Keep embeddings tensor for later use
embeddings = dataset.embeddings

In [ ]:
embeddings.shape

## 4. Train RQ-VAE

In [ ]:
import lightning as L
from torch.utils.data import DataLoader, TensorDataset

# Config for quick test
config = SemanticRQVAEConfig(
    embedding_dim=384,
    hidden_dim=256,
    codebook_size=64,  # Smaller for quick test
    num_quantizers=3,
)

# Create Lightning trainer module
trainer_module = RQVAETrainer(config=config, learning_rate=1e-3)

# Create dataloader
dataset = TensorDataset(embeddings)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset, batch_size=64, shuffle=False)

# Train with Lightning
trainer = L.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=1,
)

trainer.fit(trainer_module, train_loader, val_loader)
print("✓ RQ-VAE training complete")

## 5. Generate Semantic IDs

In [ ]:
# Generate semantic IDs for all items using the trained model
model = trainer_module.model
model.eval()

device = next(model.parameters()).device
with torch.no_grad():
    indices = model.get_semantic_ids(embeddings.to(device))
    semantic_strings = model.semantic_id_to_string(indices)

# Create mapping
item_to_semantic = {}
semantic_to_item = {}

for i, item in enumerate(items):
    item_id = item["id"]
    sem_id = semantic_strings[i]
    item_to_semantic[item_id] = {
        "codes": indices[i].cpu().tolist(),
        "semantic_id": sem_id,
    }
    semantic_to_item[sem_id] = item_id

# Save mapping
mapping = {
    "item_to_semantic": item_to_semantic,
    "semantic_to_item": semantic_to_item,
    "config": {"num_quantizers": config.num_quantizers, "codebook_size": config.codebook_size}
}

with open("data/semantic_ids.json", "w") as f:
    json.dump(mapping, f, indent=2)

# Show examples
print("\nExample semantic IDs:")
for i in range(5):
    print(f"  {items[i]['title'][:30]:30} -> {semantic_strings[i]}")

print(f"\n✓ Generated {len(semantic_to_item)} unique semantic IDs")

## 6. Generate Training Data

In [ ]:
# Generate query -> semantic ID training pairs
query_templates = [
    "Find me a {title}",
    "I'm looking for {title}",
    "Recommend a {title}",
    "{title}",
    "{description}",
]

training_examples = []
system_prompt = "You are a recommendation system. Given a user query, output the semantic ID of the most relevant item."

for item in items:
    item_id = item["id"]
    sem_id = item_to_semantic[item_id]["semantic_id"]
    
    for template in random.sample(query_templates, 2):
        query = template.format(title=item["title"], description=item["description"])
        
        training_examples.append({
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query},
                {"role": "assistant", "content": sem_id},
            ],
            "item_id": item_id,
        })

# Shuffle and split
random.shuffle(training_examples)
split_idx = int(len(training_examples) * 0.9)
train_examples = training_examples[:split_idx]
val_examples = training_examples[split_idx:]

# Save
with open("data/train.jsonl", "w") as f:
    for ex in train_examples:
        f.write(json.dumps(ex) + "\n")

with open("data/val.jsonl", "w") as f:
    for ex in val_examples:
        f.write(json.dumps(ex) + "\n")

print(f"✓ Generated {len(train_examples)} train, {len(val_examples)} val examples")
print(f"\nExample:")
print(f"  Query: {train_examples[0]['messages'][1]['content'][:50]}...")
print(f"  Target: {train_examples[0]['messages'][2]['content']}")

## 7. Fine-tune LLM with Unsloth

In [ ]:
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os

# Disable wandb for this test
os.environ["WANDB_DISABLED"] = "true"

# Use a small model for testing
BASE_MODEL = "unsloth/Qwen2.5-0.5B"  # Very small for quick test
MAX_SEQ_LENGTH = 256

print(f"Loading {BASE_MODEL}...")
llm_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

print("✓ Model loaded")

In [ ]:
# Add semantic ID tokens
semantic_tokens = []
for q in range(config.num_quantizers):
    for c in range(config.codebook_size):
        semantic_tokens.append(f"[SEM_{q}_{c}]")

tokenizer.add_special_tokens({"additional_special_tokens": semantic_tokens})
llm_model.resize_token_embeddings(len(tokenizer))

print(f"✓ Added {len(semantic_tokens)} semantic tokens")

In [ ]:
# Add LoRA adapters
llm_model = FastLanguageModel.get_peft_model(
    llm_model,
    r=8,  # Small for quick test
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✓ LoRA adapters added")

In [ ]:
# Prepare dataset
def formatting_func(examples):
    output_texts = []
    for messages in examples["messages"]:
        text = ""
        for msg in messages:
            role = msg["role"]
            content = msg["content"]
            if role == "system":
                text += f"<|system|>\n{content}\n"
            elif role == "user":
                text += f"<|user|>\n{content}\n"
            elif role == "assistant":
                text += f"<|assistant|>\n{content}\n"
        output_texts.append(text)
    return output_texts

train_dataset = Dataset.from_list(train_examples)
val_dataset = Dataset.from_list(val_examples)

print(f"✓ Dataset prepared: {len(train_dataset)} train, {len(val_dataset)} val")

In [ ]:
# Training arguments (quick test settings)
training_args = TrainingArguments(
    output_dir="checkpoints/llm",
    num_train_epochs=1,  # Just 1 epoch for test
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    logging_steps=10,
    save_strategy="no",  # Don't save for test
    eval_strategy="no",
    fp16=True,
    optim="adamw_8bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=llm_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=training_args,
    formatting_func=formatting_func,
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
)

print("✓ Trainer configured")

In [ ]:
# Train!
print("Starting training...")
trainer.train()
print("✓ Training complete!")

## 8. Test Inference

In [ ]:
# Set to inference mode
FastLanguageModel.for_inference(llm_model)

def generate_recommendation(query: str) -> str:
    system_prompt = "You are a recommendation system. Given a user query, output the semantic ID of the most relevant item."
    prompt = f"<|system|>\n{system_prompt}\n<|user|>\n{query}\n<|assistant|>\n"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(llm_model.device)
    
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=32,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
    )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Extract part after <|assistant|>
    if "<|assistant|>" in generated:
        result = generated.split("<|assistant|>")[-1].strip()
    else:
        result = generated[len(prompt):].strip()
    
    return result

print("✓ Inference mode ready")

In [ ]:
# Test queries
test_queries = [
    "Find me a premium electronics widget",
    "I need a budget sports accessory",
    "Recommend a luxury home device",
]

print("\n" + "="*60)
print("INFERENCE TESTS")
print("="*60)

for query in test_queries:
    result = generate_recommendation(query)
    
    # Try to find the item
    import re
    sem_id_match = re.search(r"(\[SEM_\d+_\d+\])+", result)
    
    print(f"\nQuery: {query}")
    print(f"Generated: {result[:100]}")
    
    if sem_id_match:
        sem_id = sem_id_match.group(0)
        if sem_id in semantic_to_item:
            item_id = semantic_to_item[sem_id]
            item = next(i for i in items if i["id"] == item_id)
            print(f"Matched: {item['title']}")
        else:
            print(f"Semantic ID not in catalogue: {sem_id}")
    else:
        print("No valid semantic ID found")

print("\n" + "="*60)
print("✓ All tests complete!")
print("="*60)

## Summary

This notebook tested:

1. **RQ-VAE Training** - ✓ Model learns to encode items as discrete codes
2. **Semantic ID Generation** - ✓ Each item gets a unique semantic ID
3. **Training Data Creation** - ✓ Query → Semantic ID pairs generated
4. **LLM Fine-tuning** - ✓ Model learns to predict semantic IDs
5. **Inference** - ✓ Model generates semantic IDs for new queries

### Next Steps

For production use:
- Use larger model (Qwen3-4B or Ministral-3B)
- Train for more epochs (3-5)
- Use full catalogue
- Deploy to Modal for serverless inference